In [3]:
"""
train_regressor_auto.py

- 지정한 폴더 안의 모든 CSV 파일을 자동으로 불러온다.
- 데이터를 합쳐서 회귀 모델(RandomForestRegressor)을 학습한다.
- 학습된 모델을 joblib으로 저장한다.
"""

import os
import glob
import joblib
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score


# ==============================
# 1. 설정값
# ==============================

# ✅ 네가 CSV를 모아둔 폴더 경로로 바꿔라
DATA_DIR = "."   # 예: "ml/data/right_curl_sets"

# ✅ CSV 파일 패턴 (일반적으로 *.csv면 충분)
CSV_PATTERN = "set_features_with_rule_scores*.csv"

# ✅ 저장할 모델 경로
MODEL_PATH = "ml/model/right_curl_score_regressor_auto.joblib"

# ✅ 피처 / 타겟 컬럼 이름 설정
#   예시: set_id,rep_id,ts,elbow_angle,stage,label 이런 CSV라면
#   - 피처: ts, elbow_angle 정도만 쓰거나, elbow_angle 하나만 써도 됨
#   - 타겟: label (점수/레이블)
FEATURE_COLS = ["rep_count", "mean_rom", "mean_tempo", "bad_frame_ratio"]   # << 필요하면 ts, stage 등 추가
TARGET_COL = "rule_score"             # << 점수/레이블 컬럼명


# ==============================
# 2. 데이터 로딩 함수
# ==============================

def load_all_csv(data_dir: str, pattern: str = "*.csv") -> pd.DataFrame:
    """
    data_dir 안의 pattern에 매칭되는 모든 CSV를 읽어서 하나의 DataFrame으로 반환
    """
    search_pattern = os.path.join(data_dir, pattern)
    csv_files = sorted(glob.glob(search_pattern))

    print("===================================")
    print(f"[INFO] 검색 패턴: {search_pattern}")
    print(f"[INFO] 발견한 CSV 파일 수: {len(csv_files)}")
    for f in csv_files:
        print(f"  - {f}")
    print("===================================")

    if len(csv_files) == 0:
        raise RuntimeError(
            f"⚠️ CSV 파일을 하나도 찾지 못했습니다. 경로/패턴을 확인하세요. (data_dir={data_dir})"
        )

    dfs = []
    for path in csv_files:
        try:
            df = pd.read_csv(path)
            dfs.append(df)
        except Exception as e:
            print(f"[WARN] 파일 읽기 실패: {path} ({e})")

    if len(dfs) == 0:
        raise RuntimeError("⚠️ CSV 파일을 읽었지만, 유효한 DataFrame이 없습니다.")

    data = pd.concat(dfs, ignore_index=True)
    print(f"[INFO] 최종 데이터 크기: {data.shape}")  # (rows, cols)
    return data


# ==============================
# 3. 피처/타겟 분리
# ==============================

def build_X_y(df: pd.DataFrame):
    """
    DataFrame에서 FEATURE_COLS, TARGET_COL을 이용해 X, y를 생성
    """
    # 컬럼 존재 여부 체크
    missing = [c for c in FEATURE_COLS + [TARGET_COL] if c not in df.columns]
    if missing:
        raise ValueError(f"⚠️ DataFrame에 아래 컬럼이 없습니다: {missing}")

    X = df[FEATURE_COLS].values.astype(np.float32)
    y = df[TARGET_COL].values.astype(np.float32)

    print("[INFO] X shape:", X.shape)
    print("[INFO] y shape:", y.shape)
    return X, y


# ==============================
# 4. 모델 학습 함수
# ==============================

def train_and_evaluate(X, y, model_path: str):
    """
    - 데이터 양이 충분하면 train/test로 나눠서 평가
    - 너무 적으면 전체를 통으로 학습만 수행
    - 모델을 model_path에 저장
    """
    n_samples = X.shape[0]
    print(f"[INFO] 샘플 수: {n_samples}")

    # 샘플 수가 10개 미만이면 굳이 train/test 나누지 말고 통으로 학습
    MIN_SPLIT_SAMPLES = 10

    if n_samples < MIN_SPLIT_SAMPLES:
        print(f"[경고] 샘플 수 {n_samples} < {MIN_SPLIT_SAMPLES} → train/test 나누지 않고 전체로 학습만 진행합니다.")
        X_train, y_train = X, y
        X_test, y_test = None, None
    else:
        X_train, X_test, y_train, y_test = train_test_split(
            X,
            y,
            test_size=0.2,
            random_state=42,
        )
        print("[INFO] Train size:", X_train.shape[0])
        print("[INFO] Test size:", X_test.shape[0])

    # RandomForest 회귀 모델 정의 (하이퍼파라미터는 필요시 조정)
    model = RandomForestRegressor(
        n_estimators=200,
        max_depth=None,
        random_state=42,
        n_jobs=-1,
    )

    print("[INFO] 모델 학습 시작...")
    model.fit(X_train, y_train)
    print("[INFO] 모델 학습 완료.")

    # 평가 (테스트셋이 있을 때만)
    if X_test is not None:
        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print("===================================")
        print("[평가] Test MAE:", mae)
        print("[평가] Test R² :", r2)
        print("===================================")
    else:
        # 전체 데이터로 학습만 했을 경우, 참고용으로 전체에 대해 MAE 계산
        y_pred = model.predict(X)
        mae = mean_absolute_error(y, y_pred)
        print("===================================")
        print("[참고] 전체 데이터 기준 MAE:", mae)
        print("===================================")

    # 모델 저장
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    joblib.dump(model, model_path)
    print(f"[INFO] 모델 저장 완료: {model_path}")


# ==============================
# 5. main
# ==============================

def main():
    # 1) CSV 로딩
    df = load_all_csv(DATA_DIR, CSV_PATTERN)

    # 2) X, y 구성
    X, y = build_X_y(df)

    # 3) 모델 학습 + 평가 + 저장
    train_and_evaluate(X, y, MODEL_PATH)


if __name__ == "__main__":
    main()


[INFO] 검색 패턴: .\set_features_with_rule_scores*.csv
[INFO] 발견한 CSV 파일 수: 1
  - .\set_features_with_rule_scores.csv
[INFO] 최종 데이터 크기: (9, 13)
[INFO] X shape: (9, 4)
[INFO] y shape: (9,)
[INFO] 샘플 수: 9
[경고] 샘플 수 9 < 10 → train/test 나누지 않고 전체로 학습만 진행합니다.
[INFO] 모델 학습 시작...
[INFO] 모델 학습 완료.
[참고] 전체 데이터 기준 MAE: 0.09333333333333337
[INFO] 모델 저장 완료: ml/model/right_curl_score_regressor_auto.joblib


In [4]:
import pandas as pd
import joblib

df = pd.read_csv("set_features_with_rule_scores.csv")

FEATURE_COLS = ["rep_count", "mean_rom", "mean_tempo", "bad_frame_ratio"]
X = df[FEATURE_COLS]

model = joblib.load("ml/model/right_curl_score_regressor_auto.joblib")
pred = model.predict(X)

print(df[["rep_count", "mean_rom", "mean_tempo", "bad_frame_ratio", "rule_score"]])
print("예측:", pred)
print("실제:", df["rule_score"].values)


   rep_count    mean_rom  mean_tempo  bad_frame_ratio  rule_score
0          7  131.117815    3.581857              1.0         1.0
1          5  116.398821    7.008600              1.0         1.0
2          3  120.220054    8.089667              1.0         1.0
3         12  162.119779    3.487667              0.0         5.0
4         14  160.107044    2.605714              0.0         5.0
5         12  162.119779    3.487667              0.0         5.0
6          5  116.398821    7.008600              1.0         1.0
7          7  131.117815    3.581857              1.0         1.0
8          3  120.220054    8.089667              1.0         1.0
예측: [1.16 1.   1.   4.8  4.88 4.8  1.   1.16 1.  ]
실제: [1. 1. 1. 5. 5. 5. 1. 1. 1.]


c:\Users\EL061\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [5]:
import pandas as pd

df = pd.read_csv("set_features_with_rule_scores.csv")

print(df[["rep_count", "mean_rom", "mean_tempo", "bad_frame_ratio", "rule_score"]])

print("\n상관계수:")
print(df[["rep_count", "mean_rom", "mean_tempo", "bad_frame_ratio", "rule_score"]].corr())


   rep_count    mean_rom  mean_tempo  bad_frame_ratio  rule_score
0          7  131.117815    3.581857              1.0         1.0
1          5  116.398821    7.008600              1.0         1.0
2          3  120.220054    8.089667              1.0         1.0
3         12  162.119779    3.487667              0.0         5.0
4         14  160.107044    2.605714              0.0         5.0
5         12  162.119779    3.487667              0.0         5.0
6          5  116.398821    7.008600              1.0         1.0
7          7  131.117815    3.581857              1.0         1.0
8          3  120.220054    8.089667              1.0         1.0

상관계수:
                 rep_count  mean_rom  mean_tempo  bad_frame_ratio  rule_score
rep_count         1.000000  0.956082   -0.878539        -0.928961    0.928961
mean_rom          0.956082  1.000000   -0.815291        -0.963092    0.963092
mean_tempo       -0.878539 -0.815291    1.000000         0.669300   -0.669300
bad_frame_ratio  -0.9

In [6]:
import pandas as pd

df = pd.read_csv("set_features_with_rule_scores.csv")

print(df["rule_score"].describe())
print(df["rule_score"].unique())

count    9.000000
mean     2.333333
std      2.000000
min      1.000000
25%      1.000000
50%      1.000000
75%      5.000000
max      5.000000
Name: rule_score, dtype: float64
[1. 5.]
